# **Семинар 5 — Создание конвейера CI/CD с помощью GitHub Actions**

## **Цель занятия**

На этом семинаре вы научитесь разрабатывать и настраивать базовый конвейер **CI/CD** для ML-модели с использованием **GitHub Actions**.

После занятия вы сможете:
- понимать структуру workflow-файла (`.github/workflows`);
- автоматически запускать обучение, тестирование и сборку проекта при каждом изменении кода;
- сохранять артефакты модели и подключать переменные окружения;
- проверять корректность пайплайна через интерфейс GitHub Actions.


## **Контекст**

С ростом сложности ML-проектов становится невозможным вручную отслеживать, что и как обучалось.  
CI/CD-конвейеры позволяют:
- автоматически проверять и запускать обучение при изменениях в коде,
- сохранять воспроизводимые артефакты (модели, метрики, логи),
- контролировать качество кода и ускорять интеграцию новых фич.

> 💬 **DevOps → MLOps:**  
> в ML-проектах CI/CD служит не только для тестирования кода, но и для **контроля жизненного цикла модели** — от обучения до деплоя.


## **План занятия**

1. Повторение принципов CI/CD и роли GitHub Actions  
2. Создание тестового ML-репозитория  
3. Настройка workflow для проверки кода и тестов  
4. Добавление шага обучения модели и сохранения артефактов  
5. Использование секретов и переменных окружения  
6. Визуализация пайплайна в GitHub UI  
7. Оптимизация: кэширование зависимостей  
8. Мини-исследование: влияние CI/CD на reproducibility  
9. Практическое задание


## **Подготовка окружения**

1. Создайте новый публичный репозиторий **`ml_ci_cd_demo`**.  
2. Добавьте файлы:
   - `train.py`
   - `test_model.py`
   - `requirements.txt`
3. В `train.py` создайте пример обучения:
   ```python
   from sklearn.datasets import load_iris
   from sklearn.linear_model import LogisticRegression
   import joblib

   X, y = load_iris(return_X_y=True)
   model = LogisticRegression(max_iter=200)
   model.fit(X, y)
   joblib.dump(model, "model.pkl")
   print("Model trained successfully.")


В test_model.py добавляем базовый скрипт:

In [1]:
import joblib
import os

def test_model_file_exists():
    assert os.path.exists("model.pkl") == False, "Model should not exist before training"

Создаем каталог в github.workflows:



## **Шаг 1. Создание базового workflow**

Файл: `.github/workflows/ci.yml`

```yaml
name: CI pipeline

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout code
        uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: "3.11"

      - name: Install dependencies
        run: pip install -r requirements.txt

      - name: Run tests
        run: pytest test_model.py





---

💡 **Вопрос для обсуждения**

Что произойдёт, если тесты упадут на этапе **Run tests**?

*Ответ:* пайплайн немедленно остановится, и сборка будет помечена как **failed**.  
Это гарантирует, что в `main` не попадёт нерабочий код.


## **Шаг 2. Добавление обучения и артефактов**

После тестов добавьте шаг обучения модели и выгрузку артефакта:

```yaml
      - name: Train model
        run: python train.py

      - name: Upload model artifact
        uses: actions/upload-artifact@v4
        with:
          name: trained-model
          path: model.pkl


Теперь каждая успешная сборка будет сохранять модель как артефакт, доступный через
Actions → Artifacts.


---

## **Шаг 3. Подключение секретов и переменных окружения**

1. В интерфейсе GitHub откройте:  
   → **Settings → Secrets → Actions → New repository secret**  
   и добавьте:
   - `API_KEY`
   - `MODEL_VERSION`

2. Подключите их в YAML:

```yaml
env:
  MODEL_VERSION: ${{ secrets.MODEL_VERSION }}

- name: Log version
  run: echo "Training model version $MODEL_VERSION"



---

💡 **Интерактив**

Почему нельзя хранить секреты прямо в коде?  
→ *Потому что это создаёт риск утечки. GitHub Secrets* хранит их в зашифрованном виде,  
и доступ получают только раннеры внутри workflow.


## **Шаг 4. Проверка успешности конвейера**

После коммита и push:
1. Перейдите во вкладку **Actions → CI pipeline**.  
2. Проверьте, что напротив всех шагов стоят зелёные галочки.  
3. В случае ошибки — разверните лог, чтобы найти причину.

🔍 *Совет:* используйте `print()` и `echo` для вывода отладочной информации.


## **Шаг 5. Оптимизация: кэширование зависимостей**

Добавьте блок **Cache pip**:

```yaml
- name: Cache pip
  uses: actions/cache@v4
  with:
    path: ~/.cache/pip
    key: ${{ runner.os }}-pip-${{ hashFiles('**/requirements.txt') }}
    restore-keys: |
      ${{ runner.os }}-pip-


## **Шаг 6. Визуализация пайплайна**

После успешного прогона в GitHub Actions отобразится граф зависимостей шагов.  
Обратите внимание:
- каждый **job** выполняется в отдельном контейнере Runner;
- шаги (`steps`) выполняются последовательно;
- результаты можно визуализировать в виде DAG.

💬 Для сложных ML-пайплайнов (с обучением, тестами и деплоем) обычно создают отдельные job:
- *build*,  
- *train*,  
- *deploy* (в docker-registry или cloud).